<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/1-Introduction%20to%20LLMs%20with%20OpenAI/1_2b-Easy_NL2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>Create a Simple Natural Language to SQL translator</h2>
    contributions by <b>Fabricio Q</b>
    
</div>

<br>

<div>
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/matiasquaglia"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

This is the unofficial repository for the book:
        <a href="https://amzn.to/4eanT1g"> <b>Large Language Models:</b> Apply and Implement Strategies for Large Language Models</a> (Apress).
        The book is based on the content of this repository, but the notebooks are being updated, and I am incorporating new examples and chapters.
        If you are looking for the official repository for the book, with the original notebooks, you should visit the
        <a href="https://github.com/Apress/Large-Language-Models-Projects">Apress repository</a>, where you can find all the notebooks in their original format as they appear in the book.

# SQL Generator
A sample of how to build a translator from natural language to SQL:

* GPT 35
* Panel
* OpenAI

Deeper explanations in the article: [How to Create a Natural Language to SQL Translator Using OpenAI API](https://medium.com/towards-artificial-intelligence/how-to-create-a-natural-language-to-sql-translator-using-openai-api-e1b1f72ac35a)

In [ ]:
!pip install openai==1.1.1
!pip install panel

In [ ]:
import openai
import panel as pn
#from mykeys import openai_api_key
openai.api_key="your-openai-api-key"

In [ ]:
def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [ ]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User promnopt.
    context.append({'role':'user', 'content':f"{prompt}."})
    context.append({'role':'system', 'content':f"""YOU ARE A SQL Assistant. \
    If the user ask for something that the result is not an SQL Order \
    just answer something nice and simple asking him, with maximum 10 words, for something that \
    can be solved with SQL."""})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Undate the panels to shjow the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

**Note:** The database information will be provided to the LLM using DBML, an expressive Domain-Specific Language (DSL) designed for describing database structures. [DBML](https://dbml.dbdiagram.io/docs/) is human-readable and well understood by modern Large Language Models (LLMs) as well, enabling the rich description of databases while using fewer tokens.

In [ ]:
context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to Maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

context.append( {'role':'system', 'content':"""
This is the definition of your database tables:

```dbml
# first table
Table employees {
  ID_usr int [pk]
  name string
}

#second table
Table salary {
  ID_usr int [ref: > employees.ID_usr]
  year date
  salary float
}

# third table
Table studies {
  ID int [pk]
  ID_usr int [ref: > employees.ID_usr]
  educational_level int
  Institution string
  Years date
  Speciality string
}
```
"""
})

#Creamos el panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Order your data…')
button_conversation = pn.widgets.Button(name="generate SQL")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)



In [ ]:
dashboard